In [ ]:
import os
import sys
import cv2
import json
import glob
import math
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from utils.file_dialog_utils import pick_video_cv2, pick_folder

In [ ]:
# --- LOADING & PREPARATION ---
def load_templates_with_names(folder_path):
    """
    Loads templates and keeps their filenames.
    Returns: list of (filename, image_bgr)
    """
    paths = sorted(glob.glob(os.path.join(folder_path, "*.png")))
    template_data = []
    for p in paths:
        img = cv2.imread(p)
        if img is not None:
            filename = os.path.basename(p)
            template_data.append((filename, img))
    return template_data


def preprocess_templates_for_video(template_data):
    """
    Converts BGR templates to Grayscale ONCE to speed up video processing.
    Returns: list of (name, gray_image, width, height)
    """
    processed = []
    for name, img_bgr in template_data:
        gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        h, w = gray.shape
        processed.append((name, gray, w, h))
    return processed


def load_geometry(geometry_path: str) -> dict:
    with open(geometry_path, "r") as f:
        geom = json.load(f)

    cx = float(geom["ellipse"]["cx"])
    cy = float(geom["ellipse"]["cy"])
    rx = float(geom["ellipse"]["rx"])
    ry = float(geom["ellipse"]["ry"])
    rot_deg = float(geom["ellipse"]["rotation_deg"])
    zero_angle_deg = float(geom["zero_angle_deg"])

    outer_inward_pct = float(geom["halo"]["outer_inward_pct"])
    inner_inward_pct = float(geom["halo"]["inner_inward_pct"])

    outer_r_norm = 1.0 - outer_inward_pct
    inner_r_norm = 1.0 - inner_inward_pct

    return dict(
        cx=cx, cy=cy, rx=rx, ry=ry,
        ellipse_rot_deg=rot_deg,
        zero_angle_deg=zero_angle_deg,
        outer_r_norm=outer_r_norm,
        inner_r_norm=inner_r_norm,
        raw=geom
    )

In [ ]:
# --- TEMPLATE MATCHING LOGIC ---
def find_best_match(target_gray, gray_templates):
    """
    Optimized matcher for video. 
    Accepts PRE-CONVERTED gray templates to save time.
    """
    best_match = {
        "score": -1.0,
        "location": None,
        "width": 0,
        "height": 0,
        "name": ""
    }

    for name, tmpl_gray, w, h in gray_templates:
        res = cv2.matchTemplate(target_gray, tmpl_gray, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

        if max_val > best_match["score"]:
            best_match["score"] = max_val
            best_match["location"] = max_loc
            best_match["width"] = w
            best_match["height"] = h
            best_match["name"] = name

    return best_match


def compute_zero_angle_for_frame(frame_bgr, gray_templates, geom: dict, min_score: float = 0.7):
    """
    Uses template matching to find the 0-pocket template in this frame,
    then returns zero_angle_deg for THIS frame (dynamic).
    """
    target_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

    best = find_best_match(target_gray, gray_templates)
    if best["location"] is None or best["score"] < min_score:
        raise RuntimeError(f"Zero template match too weak: score={best['score']:.3f}")

    tlx, tly = best["location"]
    w, h = best["width"], best["height"]

    zx = tlx + w / 2.0
    zy = tly + h / 2.0

    z_ang = ball_angle_deg(zx, zy, geom["cx"], geom["cy"])
    return z_ang, (zx, zy), best


In [ ]:
# --- SHAPE FITTING & VIDEO PROCESSING ---
def fit_ellipse(points_xy):
    """
    Fit an ellipse to (x,y) points using OpenCV.
    Needs >= 5 points.
    Returns: (center(x,y), axes(rx, ry), angle_deg)
    """
    pts = np.array(points_xy, dtype=np.float32).reshape(-1, 1, 2)
    (cx, cy), (MA, ma), angle = cv2.fitEllipse(pts)
    rx, ry = MA / 2.0, ma / 2.0
    return (float(cx), float(cy)), (float(rx), float(ry)), float(angle)


def distributed_frame_indices(total_frames: int, n: int):
    """
    Returns n indices spread across [0, total_frames-1].
    Ensures uniqueness and sorted order.
    """
    n = int(max(1, n))
    if total_frames <= 0:
        return [0]
    idx = np.linspace(0, max(0, total_frames-1), n, dtype=int)
    idx = np.unique(idx)
    return idx.tolist()


def process_video_sampled_frames(
    video_path: str,
    template_folder: str,
    n_samples: int = 30,
    min_score: float = 0.60,
    outer_inward_pct: float = 0.10,
    inner_inward_pct: float = 0.30,
):
    """
    Simplified pipeline:
      1) Sample N distributed frames from the video
      2) Template-match on ONLY those frames
      3) Collect accepted center points
      4) Fit ellipse (>=5 points) else circle fallback
      5) Overlay the fitted shape on the FIRST frame

    Params:
      - n_samples: how many distributed frames to evaluate (higher = better fit, slower)
      - min_score: NCC threshold
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Grab the first frame for overlay output
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ok, first_frame = cap.read()
    if not ok:
        cap.release()
        raise RuntimeError("Could not read first frame.")

    # Load templates
    templates_with_names = load_templates_with_names(template_folder)
    gray_templates = preprocess_templates_for_video(templates_with_names)

    sample_idxs = distributed_frame_indices(total_frames, n_samples)
    print(f"Total frames: {total_frames} | sampling {len(sample_idxs)} frames")

    rows = []

    for idx in sample_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ok, frame = cap.read()
        if not ok:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        match = find_best_match(gray, gray_templates)

        if match.get("location") is None:
            continue

        score = float(match["score"])
        if score < min_score:
            continue

        # center coordinate
        tlx, tly = match["location"]
        w, h = match["width"], match["height"]
        cx = tlx + w / 2.0
        cy = tly + h / 2.0

        rows.append({
            "frame_idx": int(idx),
            "center_x": float(cx),
            "center_y": float(cy),
            "score": score,
            "template": match.get("name", "")
        })

    cap.release()

    print(f"Accepted points: {len(rows)}")
    if not rows:
        print("No points accepted. Try lowering min_score or increasing n_samples.")
        return
    
    points_xy = [(r["center_x"], r["center_y"]) for r in rows]

    # Fit ellipse if possible, else circle fallback
    overlay = first_frame.copy()

    if len(points_xy) >= 5:
        (cx, cy), (rx, ry), angle = fit_ellipse(points_xy)
        cv2.ellipse(
            overlay,
            (int(cx), int(cy)),
            (int(rx), int(ry)),
            angle,
            0, 360,
            (0, 0, 255),
            3
        )
        cv2.circle(overlay, (int(cx), int(cy)), 4, (0, 255, 0), -1)

        print(f"\nEllipse -> center=({cx:.1f},{cy:.1f}), axes=({rx:.1f},{ry:.1f}), angle={angle:.1f}°")

        # --- define zero reference angle using FIRST frame (stable reference) ---
        first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
        zero_match = find_best_match(first_gray, gray_templates)

        if zero_match.get("location") is None:
            raise RuntimeError("Could not find zero template on first frame; cannot compute zero reference angle.")

        tlx, tly = zero_match["location"]
        w, h = zero_match["width"], zero_match["height"]
        zero_x = tlx + w / 2.0
        zero_y = tly + h / 2.0

        dx0 = zero_x - cx
        dy0 = zero_y - cy

        # angle in image space (y axis inverted)
        zero_angle_deg = (np.degrees(np.arctan2(-dy0, dx0)) % 360.0)

        print(f"Zero reference on first frame -> (x,y)=({zero_x:.1f},{zero_y:.1f}) angle={zero_angle_deg:.2f}° score={zero_match['score']:.3f}")

        # export geometry handoff artifact
        geometry = {
            "ellipse": {
                "cx": float(cx),
                "cy": float(cy),
                "rx": float(rx),
                "ry": float(ry),
                "rotation_deg": float(angle),
            },
            "zero_angle_deg": float(zero_angle_deg),
            "halo": {
                "outer_inward_pct": float(outer_inward_pct), 
                "inner_inward_pct": float(inner_inward_pct)
            },
            "wheel": {
                "type": "EU",
                "pockets": 37
            }
        }

        with open("wheel_geometry.json", "w") as f:
            json.dump(geometry, f, indent=2)

In [ ]:
# --- GEOMETRY & INFERENCE ---

def ensure_geometry(
    setup_video_path: str,
    templates_dir: str,
    geometry_path: str | None,
    n_samples: int,
    min_score: float,
    outer_inward_pct: float,
    inner_inward_pct: float,
    output_path_if_built: str = "wheel_geometry.json",
) -> str:
    """
    If geometry_path is provided and exists, use it.
    Otherwise build geometry and return the path of the newly written JSON.
    """
    if geometry_path and os.path.isfile(geometry_path):
        print(f"Using existing geometry: {geometry_path}")
        return geometry_path

    print("No geometry provided (or file missing). Building geometry now...")
    process_video_sampled_frames(
        video_path=setup_video_path,
        template_folder=templates_dir,
        n_samples=n_samples,
        min_score=min_score,
        outer_inward_pct=outer_inward_pct,
        inner_inward_pct=inner_inward_pct
    )
    # ^ your function currently writes "wheel_geometry.json" in cwd (based on your calls)

    if not os.path.isfile(output_path_if_built):
        raise FileNotFoundError(f"Expected geometry file was not created: {output_path_if_built}")

    return output_path_if_built


EUROPEAN_WHEEL_ORDER = [
    0,
    32, 15, 19, 4, 21, 2, 25, 17, 34,
    6, 27, 13, 36, 11, 30, 8, 23, 10,
    5, 24, 16, 33, 1, 20, 14, 31,
    9, 22, 18, 29, 7, 28, 12,
    35, 3, 26
]

AMERICAN_WHEEL_ORDER = [
    0, 28, 9, 26, 30, 11, 7, 20, 32, 17,
    5, 22, 34, 15, 3, 24, 36, 13, 1, "00",
    27, 10, 25, 29, 12, 8, 19, 31, 18,
    6, 21, 33, 16, 4, 23, 35, 14, 2
]

def get_wheel_order(wheel_type: str):
    wheel_type = wheel_type.upper().strip()
    if wheel_type in ("EU", "EUROPE", "EUROPEAN"):
        return EUROPEAN_WHEEL_ORDER
    if wheel_type in ("US", "USA", "AMERICAN"):
        return AMERICAN_WHEEL_ORDER
    raise ValueError("wheel_type must be 'EU' or 'US'")


def rotate_point_to_ellipse_axes(x, y, cx, cy, rot_deg):
    """Rotate (x,y) around (cx,cy) by -rot_deg so ellipse becomes axis-aligned."""
    theta = math.radians(rot_deg)
    tx, ty = x - cx, y - cy
    cos_t, sin_t = math.cos(theta), math.sin(theta)
    xr =  tx * cos_t + ty * sin_t
    yr = -tx * sin_t + ty * cos_t
    return xr, yr


def ball_angle_deg(x, y, cx, cy) -> float:
    """0° = +x to the right, CCW positive (image y goes down so we flip)."""
    dx = x - cx
    dy = y - cy
    return (math.degrees(math.atan2(-dy, dx)) % 360.0)


def r_norm_in_ellipse_space(x, y, geom: dict) -> float:
    xr, yr = rotate_point_to_ellipse_axes(x, y, geom["cx"], geom["cy"], geom["ellipse_rot_deg"])
    # normalized radius in ellipse coordinates
    return math.sqrt((xr / geom["rx"])**2 + (yr / geom["ry"])**2)


def is_point_in_halo(x, y, geom: dict) -> bool:
    r = r_norm_in_ellipse_space(x, y, geom)
    return (geom["inner_r_norm"] <= r <= geom["outer_r_norm"])


def angle_to_pocket_index_using_zero_center(ball_ang_deg: float, zero_center_deg: float, n_pockets: int) -> int:
    """
    Assumes zero_center_deg points to CENTER of pocket 0.
    Converts to a boundary reference by subtracting half a sector.
    """
    sector = 360.0 / n_pockets
    zero_boundary = (zero_center_deg - 0.5 * sector) % 360.0
    rel = (ball_ang_deg - zero_boundary) % 360.0
    idx = int(rel // sector)
    return max(0, min(n_pockets - 1, idx))


def pocket_value_from_angle(ball_ang_deg: float, zero_center_deg: float, wheel_order):
    idx = angle_to_pocket_index_using_zero_center(ball_ang_deg, zero_center_deg, n_pockets=len(wheel_order))
    return idx, wheel_order[idx]


def compute_zero_angle_for_frame(frame_bgr, gray_templates, geom: dict, min_score: float = 0.7):
    target_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

    best = find_best_match(target_gray, gray_templates)  # your existing helper
    if best["location"] is None or best["score"] < min_score:
        raise RuntimeError(f"Zero template match too weak: score={best['score']:.3f}")

    tlx, tly = best["location"]
    w, h = best["width"], best["height"]
    zx = tlx + w / 2.0
    zy = tly + h / 2.0

    z_ang = ball_angle_deg(zx, zy, geom["cx"], geom["cy"])
    return z_ang, (zx, zy), best


def run_ball_landing_inference(
    video_path: str,
    model_path: str,
    geom: dict,
    gray_templates,
    wheel_type: str = "EU",
    conf_thres: float = 0.35,
    iou_thres: float = 0.70,
    device=0,
    ball_class_id: int = 0,
    landing_seconds_required: float = 2.5,
    min_zero_score: float = 0.70
):
    """
    Runs YOLO inference on the video to detect when the ball has landed in the halo.
    Uses the geometry to determine if detected ball is in the halo for a sustained period.
    """
    wheel_order = get_wheel_order(wheel_type)

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    cap.release()

    frames_required = int(round(landing_seconds_required * fps))
    model = YOLO(str(model_path))

    consecutive = 0
    landed = False
    landing_frame_idx = None
    landing_ball_xy = None
    landing_ball_angle = None
    landing_pocket_idx = None
    landing_pocket_value = None

    frame_idx = -1

    for r in model.predict(
        source=str(video_path),
        stream=True,
        conf=conf_thres,
        iou=iou_thres,
        device=device
    ):
        frame_idx += 1

        if r.boxes is None or len(r.boxes) == 0:
            consecutive = 0
            continue

        # pick best ball detection (class id + highest conf)
        best = None
        for b in r.boxes:
            cls_id = int(b.cls[0])
            if cls_id != ball_class_id:
                continue
            conf = float(b.conf[0])
            if best is None or conf > best["conf"]:
                x1, y1, x2, y2 = map(float, b.xyxy[0])
                best = dict(conf=conf, xyxy=(x1, y1, x2, y2))

        if best is None:
            consecutive = 0
            continue

        x1, y1, x2, y2 = best["xyxy"]
        bx = (x1 + x2) / 2.0
        by = (y1 + y2) / 2.0

        in_halo = is_point_in_halo(bx, by, geom)

        if in_halo:
            consecutive += 1
        else:
            consecutive = 0

        if (not landed) and consecutive >= frames_required:
            landed = True
            landing_frame_idx = frame_idx
            landing_ball_xy = (bx, by)
            
            landing_ball_angle = ball_angle_deg(bx, by, geom["cx"], geom["cy"])

            # --- NEW: read the landing frame from the video ---
            cap2 = cv2.VideoCapture(str(video_path))
            cap2.set(cv2.CAP_PROP_POS_FRAMES, landing_frame_idx)
            ok2, landing_frame = cap2.read()
            cap2.release()
            if not ok2:
                raise RuntimeError(f"Could not read landing frame {landing_frame_idx} to compute dynamic zero angle.")

            # --- NEW: compute ZERO angle on the landing frame (dynamic) ---
            zero_angle_landing, zero_xy, zero_match = compute_zero_angle_for_frame(
                landing_frame, gray_templates, geom, min_score=min_zero_score
            )

            # --- NEW: compute pocket using dynamic zero (center-based) ---
            landing_pocket_idx = angle_to_pocket_index_using_zero_center(
                landing_ball_angle, zero_angle_landing, n_pockets=len(wheel_order)
            )
            landing_pocket_value = wheel_order[landing_pocket_idx]

            # (optional) store debug
            landing_zero_angle = zero_angle_landing
            landing_zero_xy = zero_xy
            landing_zero_score = float(zero_match["score"])

            break


    return dict(
        landed=landed,
        landing_frame_idx=landing_frame_idx,
        landing_ball_xy=landing_ball_xy,
        landing_ball_angle=landing_ball_angle,
        landing_pocket_idx=landing_pocket_idx,
        landing_pocket_value=landing_pocket_value,
        wheel_type=wheel_type,
        frames_required=frames_required,
        zero_angle_landing=landing_zero_angle if landed else None,
        zero_xy=landing_zero_xy if landed else None,
        zero_score=landing_zero_score if landed else None,
    )

In [ ]:
#--- DEBUG VISUALIZATION ---
def debug_draw_ellipse_from_geometry(
    video_path: str, 
    geometry_path: str, 
    frame_index: int = 0
):
    """
    Visual sanity check:
    - Draw fitted ellipse
    - Draw center point
    - Draw zero-angle ray
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Failed to open video: {video_path}")

    # seek to frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ok, frame = cap.read()
    cap.release()
    if not ok:
        raise RuntimeError(f"Failed to read frame {frame_index}")

    with open(geometry_path, "r") as f:
        geom = json.load(f)

    cx = geom["ellipse"]["cx"]
    cy = geom["ellipse"]["cy"]
    rx = geom["ellipse"]["rx"]
    ry = geom["ellipse"]["ry"]
    rot = geom["ellipse"]["rotation_deg"]
    zero_angle = geom["zero_angle_deg"]

    overlay = frame.copy()

    # draw fitted ellipse
    cv2.ellipse(
        overlay,
        (int(cx), int(cy)),
        (int(rx), int(ry)),
        rot,
        0, 360,
        (0, 255, 0),
        2
    )

    # draw center
    cv2.circle(overlay, (int(cx), int(cy)), 4, (0, 255, 0), -1)

    # draw zero-angle ray
    r = int(max(rx, ry) * 1.1)
    x2 = int(cx + r * np.cos(np.radians(zero_angle)))
    y2 = int(cy - r * np.sin(np.radians(zero_angle)))  # minus because image y-axis
    cv2.line(overlay, (int(cx), int(cy)), (x2, y2), (0, 255, 255), 2)

    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Ellipse Sanity Check")
    plt.show()

    
def debug_draw_halo_from_geometry(
    video_path: str,
    geometry_path: str,
    frame_index: int = 0,
    show_filled_band: bool = True,
    band_alpha: float = 0.35
):
    """
    Visual sanity check:
    - Draw fitted ellipse
    - Draw halo outer+inner boundary ellipses (computed from inward %)
    - Optionally render the halo band as a filled mask overlay
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Failed to open video: {video_path}")

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ok, frame = cap.read()
    cap.release()
    if not ok:
        raise RuntimeError(f"Failed to read frame {frame_index}")

    with open(geometry_path, "r") as f:
        geom = json.load(f)

    cx = float(geom["ellipse"]["cx"])
    cy = float(geom["ellipse"]["cy"])
    rx = float(geom["ellipse"]["rx"])
    ry = float(geom["ellipse"]["ry"])
    rot = float(geom["ellipse"]["rotation_deg"])

    outer_inward = float(geom["halo"]["outer_inward_pct"])
    inner_inward = float(geom["halo"]["inner_inward_pct"])

    # Halo boundaries are "shrunk" versions of the fitted ellipse
    # outer boundary = slightly inside outer rim
    # inner boundary = further inside
    outer_rx = rx * (1.0 - outer_inward)
    outer_ry = ry * (1.0 - outer_inward)

    inner_rx = rx * (1.0 - inner_inward)
    inner_ry = ry * (1.0 - inner_inward)

    overlay = frame.copy()

    # Draw fitted ellipse (baseline)
    cv2.ellipse(overlay, (int(cx), int(cy)), (int(rx), int(ry)), rot, 0, 360, (0, 255, 0), 2)

    # Draw halo boundaries
    cv2.ellipse(overlay, (int(cx), int(cy)), (int(outer_rx), int(outer_ry)), rot, 0, 360, (255, 255, 0), 2)  # outer halo line
    cv2.ellipse(overlay, (int(cx), int(cy)), (int(inner_rx), int(inner_ry)), rot, 0, 360, (0, 255, 255), 2)  # inner halo line

    # Optional: filled band mask for the halo region
    if show_filled_band:
        mask_outer = np.zeros(frame.shape[:2], dtype=np.uint8)
        mask_inner = np.zeros(frame.shape[:2], dtype=np.uint8)

        cv2.ellipse(mask_outer, (int(cx), int(cy)), (int(outer_rx), int(outer_ry)), rot, 0, 360, 255, -1)
        cv2.ellipse(mask_inner, (int(cx), int(cy)), (int(inner_rx), int(inner_ry)), rot, 0, 360, 255, -1)

        band_mask = cv2.subtract(mask_outer, mask_inner)

        # Paint the band on a colored layer (red-ish) then alpha blend
        color_layer = np.zeros_like(frame, dtype=np.uint8)
        color_layer[band_mask > 0] = (0, 0, 255)

        overlay = cv2.addWeighted(overlay, 1.0, color_layer, band_alpha, 0)

    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Halo Sanity Check (outer/inner boundaries + optional band)")
    plt.show()

In [ ]:
# --- MAIN EXECUTION ---

# 1) Pick inputs
_, VIDEO_PATH = pick_video_cv2(title="Select Input Video for Main Execution")
_, setup_video_path = pick_video_cv2(title="Select Input Video for Wheel Geometry Setup")
templates_dir = pick_folder(title="Select Templates Folder")

MODEL_PATH = r"C:\Users\Gabriel\Documents\Dissertation\Code\models\yolo\RD2_Model.pt"

# Wheel geometry setup parameters
N_SAMPLES = 10
MIN_SCORE = 0.80
OUTER_INWARD_PCT = 0.10
INNER_INWARD_PCT = 0.30

# Inference parameters for the main execution
CONF_THRES = 0.35
IOU_THRES  = 0.70

# Optional
GEOMETRY_PATH = None
# GEOMETRY_PATH = r"C:\Users\Gabriel\Documents\Dissertation\Code\notebooks\roulette_cv\wheel_geometry.json

# 2) Ensure geometry exists
geometry_path = ensure_geometry(
    setup_video_path=setup_video_path,
    templates_dir=templates_dir,
    geometry_path=GEOMETRY_PATH,
    n_samples=N_SAMPLES,
    min_score=MIN_SCORE,
    outer_inward_pct=OUTER_INWARD_PCT,
    inner_inward_pct=INNER_INWARD_PCT,
)

geom = load_geometry(geometry_path)

# 3) Inference
result = run_ball_landing_inference(
    video_path=VIDEO_PATH,
    model_path=MODEL_PATH,
    geom=geom,
    wheel_type="EU",
    conf_thres=CONF_THRES,
    iou_thres=IOU_THRES,
    device=0, # 0 for GPU, "cpu" for CPU
    ball_class_id=0,
    landing_seconds_required=2.5,
    gray_templates=gray_templates
)

print(result)